# Chapter 1 One Variable Optimization
This notebook serves as a way to familiarize myself with the essential functions employed during math modeling.
This is for Chapter 1 of Meerschaert

In [13]:
# imports
import math
import numpy as np
from scipy.optimize import minimize, minimize_scalar
from sympy import symbols, diff, Eq, solve
import matplotlib.pyplot as plt


## Ex 1.1 The Pig Problem

A pig weighing 200 pounds gains 5 pounds per day and costs 45 cents a day to keep. The market price for pigs is 65 cents per pound, but is falling 1 cent per day. When should the pig be sold?

We declare the following:

#### Variables:
t = time (days)

w = weight of pig (lbs)

p = price for pigs (dollar/lb)

C = cost of keeping pig t days (dollar)

R = revenue obtained by selling pig (dollar) 

P = profit from sale of pig (dollar)

#### Assumptions:
w = 200 + 5t

p = 0.65 − 0.01t C = 0.45t R=p·w P=R−C t≥0

##### Objective:
Maximize P

In [14]:
# variable declarations
t, w, p, C, R, P = symbols('t w p C R P')
w = 200 + 5 * t
p = 0.65 - 0.01 * t 
C = 0.45 * t
R = p * w
P = R - C

# We want to maximize P when x = t >= 0. define P as f(x)
x = symbols('x')
f = P.subs({t: x})
print(f)

# find the derivative of f(x)
dy_dx = diff(f, x)
print("f'(x): ", dy_dx)


-0.45*x + (0.65 - 0.01*x)*(5*x + 200)
f'(x):  0.8 - 0.1*x


Note that f'(x) achieves local maximum at 8, so $(x,y) = (8, 133.20)$ is the maximum. Hence we conclude that selling the pigs after 8 days will result in maximum profits. We can draw a graph of $y$:

In [ ]:
def y_function(x):
    return -0.45*x + (0.65 - 0.01*x)*(5*x + 200)

# Generate x values
x_values = np.linspace(0, 20, 100)  # Adjust the range as needed

# Calculate corresponding y values
y_values = y_function(x_values)

result = minimize_scalar(lambda x: -y_function(x), bounds=(0, 20), method='bounded')
x_max = result.x
y_max = -result.fun

# Plot the function
plt.plot(x_values, y_values, label=r'$y = -0.45x + (0.65 - 0.01x)(5x + 200)$')
plt.scatter(x_max, y_max, color='red', label=f'Maximum: ({x_max:.2f}, {y_max:.2f})')

plt.xlabel('x')
plt.ylabel('y')
plt.title('Net profit f(x) of the pig problem with respect to time x')
plt.legend()
plt.grid(True)
plt.show()

## 1.2 Sensitivity Analysis
Though we have solve the problems, yet the assumptions made are not always completely right, so we want to test the sensitivity of the assumptions. We continue working on ex.1.1.

We analyze the sensitivity of best sell pig time $x$ with respect to the decreasing rate of price $r$:
Set $r$ as an unknown parameter, then we have

$$p = 0.65 - rt$$

In [25]:
# replace p with p = 0.65 - rx
r = symbols('r')
f = P.subs({p: 0.65 - r * x, t: x})

# find the derivative of f(x)
df_dx = diff(f, x)
print(df_dx)

# find the point where df_dx = 0
sol = solve(df_dx, x)
print("sol: ", sol)

-5*r*x - r*(5*x + 200) + 2.8
sol:  [-20.0 + 0.28/r]


Hence when $x \geq 0$, i.e., $0 < r \leq 0.014$, then the best selling time $x$ is given by $x = -20.0+\frac{0.28}{r}$.

Now consider the growing rate $g$ of the pigs, let
$$w = 200 + gt$$
then, similarly,

In [26]:
# replace w with w = 200+gx
g = symbols('g')
y = P.subs({w: 200 + g * x, t: x})

# find the derivative of f(x)
dy_dx = diff(y, x)

# find the point where dy_dx = 0
sol_g = solve(dy_dx, x)
print("sol_g: ", sol_g)


sol:  [2.5*(13.0*g - 49.0)/g]


In this case, when $x$ calculating from sol_g is non-negative, we conclude that it gives the best selling time of pig.

### Definition: sensitivity
If a $\Delta r$ change of $r$ results in a $\Delta x$ change of $x$, then we consider the ratio of the  relative changes. Let $\Delta r \rightarrow 0$, we define the following limit to be the *sensitivity* of $x$ with respect to $r$, denoted $S(x,\:r)$.
$$S(x,\:r) = \frac{\Delta x/x}{\Delta r/r} \rightarrow \frac{dx}{dr} \cdot \frac{r}{x}$$

In the pig selling problem, when $r = 0.01$, with the calculated best selling time $x = 8$, we have 
$$\frac{dx}{dr} = \frac{-7}{25r^2} = -2800$$
so
$$S(x,r) = \frac{dx}{dr} \cdot \frac{r}{x} = (-2800) \frac{0.01}{8} = \frac{-7}{2}$$
We interpreted this result as "an increase of $2 \%$ of $r$ will results in a decrease of $7 \%$ of $x$."

Similarly, when $g = 5$,
$$\frac{dx}{dg} = \frac{245}{2g^2} = 4.9$$
so
$$S(x,g) = \frac{dx}{dg} \cdot \frac{g}{x} = (4.9) \frac{5}{8} = 3.0625$$
We interpreted this result as "an increase of $1 \%$ of $g$ will results in an increase of $3 \%$ to sell the pig."

Note that we want to make judgement before conducting the sensitivity analysis. We need to conduct analysis of those parameters with uncertainty. 


## 1.3 Sensitivity and Robustness
A mathematical model is **robust** if the conclusions it leads to remain true even though the model is not completely accurate.

Moving back to ex1.1, where we consider
$$P = pw - 0.45t$$
where $p$ is the price of pig (lb/$\$$) and $w$ the weight of pig in pounds. If the initial data of the model is not that far away from realityy, then the best selling time of pig should be given by $P'$. Taking the derivative with respect to $t$, by the Chain rule we have
$$p'w - pw' = 0.45$$
$p'w+pw'$ denotes the increasing rate of pig price. Hence the model tells us that if the price of pig is increasing faster than the raising cost, then we should not sell the pig. $p'w$ stands for the loss from the decrease of price, while $pw'$ represents the value from the increase of pig weight. However, those factors may not develop as we'd expected, so the best way to solve this issue is to raise the pigs for a longer time and then reestimate $p,p',w,w'$.